In [22]:
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D, Dropout
from keras.regularizers import l2

In [17]:
# Constants
DATA_DIR = 'data/'
CSV_FILE = 'driving_log.csv'

# Parameters
steering_correction = 0.25

In [18]:
# Load csv log file
lines = []
with open(DATA_DIR+CSV_FILE) as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # skip the headers
    for line in reader:
        lines.append(line)

In [13]:
# Load images and steering data

#Cameras parameters
cameras = ['center', 'left', 'right']
cameras_index = {'center':0, 'left':1, 'right':2} # 0:center, 1:left, 2:right
cameras_steering_corretion = {'center':0, 'left':steering_correction, 'right':-steering_correction}

# Data containers
images = []
measurements = []

# Uploading images
for line in lines:
    for camera in cameras:
        source_path = DATA_DIR + line[cameras_index[camera]].strip()
        image = cv2.imread(source_path)
        images.append(image)
        measurement = float(line[3])+cameras_steering_corretion[camera]
        measurements.append(measurement)
X_data = np.array(images)
y_data = np.array(measurements)

In [24]:
# Define simple model
model = Sequential()
model.add(Cropping2D(cropping=((30,25),(0,0)), input_shape = (160, 320, 3)))
model.add(Lambda(lambda x: x / 255.0 - 0.5))
model.add(Conv2D(24, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(36, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(48, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'same', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Flatten())
model.add(Dense(80, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(40, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(16, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(10, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dense(1, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_7 (Cropping2D)    (None, 105, 320, 3)       0         
_________________________________________________________________
lambda_8 (Lambda)            (None, 105, 320, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 51, 158, 24)       1824      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 77, 36)        21636     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 37, 48)        43248     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 5, 19, 64)         27712     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 2, 9, 64)          36928     
__________

In [25]:
# Train simple model
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_data, y_data, validation_split = 0.2, shuffle = True, epochs = 3)

Train on 19286 samples, validate on 4822 samples
Epoch 1/3
19286/19286 [==============================] - 1152s - loss: 0.0903 - val_loss: 0.0409
Epoch 2/3
19286/19286 [==============================] - 1116s - loss: 0.0381 - val_loss: 0.0350
Epoch 3/3
19286/19286 [==============================] - 1268s - loss: 0.0354 - val_loss: 0.0347


In [26]:
# Save model
model.save('model.h5')